In [2]:
import numpy
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten


Loading my dataset in.

In [7]:
X_train = numpy.loadtxt('input.csv', delimiter= ',')
Y_train = numpy.loadtxt('labels.csv', delimiter= ',')

# apparently the delimiter separates values with the specified character
X_test = numpy.loadtxt('input_test.csv', delimiter= ',')
Y_test = numpy.loadtxt('labels_test.csv', delimiter= ',')